In [16]:
import pymysql
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()
from selenium.webdriver.common.by import By
from datetime import datetime
import re
import time
browser = webdriver.Chrome()

conn=pymysql.connect(
    host='localhost',  # 데이터베이스 서버 주소
    user='root',       # 데이터베이스 사용자 이름
    password='tldkswndlssla',  # 데이터베이스 비밀번호
    db='YES24',       # 데이터베이스 이름
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:


    link_list=[]     
        #got the link of searching "python", and iterated with for statement to get the each page.
        # the goal was get the elements of books to get the links for each book in the next step.
    for page in range(1,4):
        browser.get(f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={page}&pageSize=24')
        inform = browser.find_elements(By.CLASS_NAME, 'gd_name')
    # used for statement to iterate each book element to get the each book link and add to list.
    # this will be used to get the informations such as title, publisher, etc in the next step. 

        for book_link in inform:
            link = book_link.get_attribute('href')
            link_list.append(link)
            time.sleep(0.1)

                
    for link in link_list:
        browser.get(link)


        try:
            title = browser.find_element(By.CLASS_NAME, 'gd_name').text
            publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
            publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
            author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
            rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
            rating = float(rating)
            review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
            review=int(review.replace(",",""))
            sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
            sales=int(sales.replace(",",""))
            price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
            price=int(price.replace(",",""))    
            ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[3][:-1]
            ranking = int(ranking)
            ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[3][:-1]
            ranking_weeks = int(ranking_weeks)
        except Exception as e:
            rating == 0.0
            review == 0
            sales == 0
            ranking == 0
            ranking_weeks == 0
        
            match = re.match(r'(\d+)년 (\d+)월 (\d+)일',publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing  = "2023-01-01"

        sql = """INSERT INTO BOOKS (title, author, publisher, publishing, rating, 
                                review, sales, price, ranking, ranking_weeks)
                                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
        cur.execute(sql, (title, author, publisher, publishing))
        conn.commit()


NameError: name 'ranking' is not defined

In [43]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()
from selenium.webdriver.common.by import By
from datetime import datetime
import re
import time
browser = webdriver.Chrome()



link_list=[]     
    #got the link of searching "python", and iterated with for statement to get the each page.
    # the goal was get the elements of books to get the links for each book in the next step.
for page in range(1,4):
    browser.get(f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={page}&pageSize=24')
    inform = browser.find_elements(By.CLASS_NAME, 'gd_name')
# used for statement to iterate each book element to get the each book link and add to list.
# this will be used to get the informations such as title, publisher, etc in the next step. 

    for book_link in inform:
        link = book_link.get_attribute('href')
        link_list.append(link)
        # time.sleep(0.1)

            
for link in link_list:
    browser.get(link)


    try:
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        rating = browser.find_element(By.CLASS_NAME, 'gd_rating').find_element(By.CLASS_NAME, 'yes_b').text
        rating = float(rating.replace(",",""))  
        review = browser.find_element(By.CLASS_NAME, 'gd_reviewCount').find_element(By.CLASS_NAME, 'txC_blue').text
        review=int(review.replace(",",""))
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales=int(sales.replace(",",""))
        price = browser.find_element(By.CLASS_NAME, 'nor_price').text[:-1]
        price=int(price.replace(",",""))    
        ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0].split(" ")[2][:-1]
        # ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text
    except Exception as e:
        rating == 0.0
        review == 0
        sales == 0
        ranking == 0
        # ranking_weeks == 0
        
        match = re.match(r'(\d+)년 (\d+)월 (\d+)일',publishing)
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing  = "2023-01-01"

    print(title, author, publisher, publishing, rating, review, sales, price, ranking)
    print(type(rating))
        


나는 읽고 쓰고 버린다 손웅정 저 난다 2024년 04월 20일 9.9 23 185940 15300 1
<class 'float'>
불변의 법칙 모건 하우절 저/이수경 역 서삼독 2024년 02월 28일 9.6 151 505263 22500 2
<class 'float'>
흔한남매 16 흔한남매 원저/백난도 글/유난희 그림/흔한컴퍼니 감수 미래엔아이세움 2024년 04월 25일 10.0 2 110700 13050 3
<class 'float'>
나를 소모하지 않는 현명한 태도에 관하여 마티아스 뇔케 저/이미옥 역 퍼스트펭귄 2024년 03월 10일 9.6 65 320244 16020 4
<class 'float'>
4~7세 보고 만지는 수학은 이렇게 가르칩니다 최경희 저 블루무스 2024년 04월 30일 10.0 5 88770 22500 5
<class 'float'>
삶이 흔들릴 때 뇌과학을 읽습니다 이케가야 유지 글/김성기 역 힉스 2024년 03월 27일 10.0 23 111765 16200 6
<class 'float'>
일류의 조건 사이토 다카시 저/정현 역 필름 2024년 03월 20일 9.6 217 267000 17820 7
<class 'float'>
세이노의 가르침 세이노(SayNo) 저 데이원 2023년 03월 02일 9.0 2189 1625994 6480 8
<class 'float'>
빨모쌤의 라이브 영어회화 신용하 저 웅진지식하우스 2024년 04월 24일 9.7 29 107649 19800 9
<class 'float'>
꽃길이 따로 있나, 내 삶이 꽃인 것을 오평선 저 포레스트북스 2024년 03월 22일 9.8 59 164850 15120 10
<class 'float'>
나는 메트로폴리탄 미술관의 경비원입니다 패트릭 브링리 저/김희정, 조현주 역 웅진지식하우스 2023년 11월 24일 9.2 288 425319 15750 11
<class 'float'>
던전밥 14 쿠이 료코 글그림/김민재 역 소미미디어 2024-05-03 9.

In [42]:
for i in range(5):
  try:
    raise
  except: 
    print(i)
  print('hi')

0
hi
1
hi
2
hi
3
hi
4
hi
